In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca
from helper_fns import yaml_load

In [39]:
from robot import Robot
from contact import Contact
from impedance_controller import ImpedanceController
from decision_vars import DecisionVarSet
par = {'pos': [0, 0, 0.0],
       'stiff': [0, 0, 100],
        'rest':[0, 0, 0.2]}
attrs = yaml_load('config/attrs.yaml')
attrs = {k:attrs[k] for k in ['lb', 'ub']}
c = Contact('c/', par, [], attrs=attrs)
imp = ImpedanceController(pars={'imp_stiff':[300, 300, 300], 'imp_rest':[0, 0, 0.5]}, 
                          sym_vars = ['imp_rest', 'imp_stiff'],
                          attrs = attrs)
rob = Robot('config/franka.urdf', subsys = [c], ctrl = imp, attrs = attrs)
rob.build_step(0.1)
print(rob.step_vec)

Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['c/']
  Building controller imp_ctrl
step_vec:(xi[14],imp_rest[3],imp_stiff[3],M_inv[7x7])->(xi[14]) SXFunction


In [45]:
from mpc import MPC
from helper_fns import mult_shoot_rollout

mpc = MPC(robots={'free':rob}, mpc_params={'H':20, 'dt':0.1})
params = {'init_state': ca.DM.zeros(14), 
          'belief_free':1.0, 
          'imp_stiff':200*ca.DM.ones(3),}
mpc.solve(params)

  Building controller imp_ctrl
This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4555
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1427

Total number of variables............................:      400
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      120
                     variables with only upper bounds:        0
Total number of equality constraints.................:      280
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(r

DM(
[[-0.1, -0.1, -0.0999999, -0.0999999, -0.0999999, -0.0999999, -0.0999998, -0.0999998, -0.0999998, -0.0999997, -0.0999996, -0.0999994, -0.0999989, -0.0999969, 0.0557814, 0.0999937, 0.0999955, 0.0999957, 0.0999953, -2.62757e-18], 
 [-0.0999957, -0.0999934, 0.099958, 0.0999846, 0.0999906, 0.0999935, 0.099995, 0.0999958, 0.0999959, 0.0999956, 0.099995, 0.0999931, 0.0999863, 0.0999091, -0.0833108, -0.0998543, -0.0998297, -0.099749, -0.0999131, -2.62757e-18], 
 [0.0999998, 0.0999997, 0.0999995, 0.0999991, 0.0999981, 0.099984, -0.0999967, -0.0999986, -0.0999992, -0.0999995, -0.0999996, -0.0999997, -0.0999997, -0.0999997, -0.0999997, -0.0999997, -0.0999997, -0.0999997, -0.0999997, -2.62757e-18]])